### Realização dos imports

In [ ]:
%pip install pandas numpy faker
import pandas as pd
import numpy as np
import random as r
import os
from datetime import datetime
from faker import Faker

Note: you may need to restart the kernel to use updated packages.


### Criação dos registros

In [ ]:
# iniciando a instância do Faker
fake = Faker()

# criando uma seed para random e Faker para garantir consistência na geração dos dados
r.seed(27)
Faker.seed(27)

n = 1000 # número de registros no dataset

# definindo as datas de início e fim para o dataset
data_inicio = datetime.strptime('2023-01-01', '%Y-%m-%d').date()
data_fim = datetime.strptime('2023-12-31', '%Y-%m-%d').date()

# gerando datas aleatórias dentro do limite estabelecido
datas = [fake.date_between(data_inicio, data_fim) for _ in range(n)]

# criando manualmente as categorias
lista_categorias = ['Higiene pessoal', 'Remedios', 'Produtos de beleza', 'Conveniencia']

# criando manualmente os produtos já vinculados às respectivas categorias e com os os respectivos preços
lista_produtos_categoria = {
    'Higiene pessoal': {'Escova de dente': 20.50, 'Sabonete em barra': 5.50, 'Sabonete liquido': 12.25, 'Preservativo': 2.50, 'Fio dental': 14.90},
    'Remedios': {'Ibuprofeno': 12.60, 'Paracetamol': 9.90, 'Dipirona': 7.60, 'Aspirina': 11.50, 'Naproxeno': 21.90},
    'Produtos de beleza': {'Protetor solar': 52.50, 'Hidratante labial': 19.90, 'Batom': 25.50, 'Locao para acne': 34.00, 'Acetona': 9.90},
    'Conveniencia': {'Chocolate': 8.70, 'Bala de gelatina': 11.90, 'Energetico': 15.90, 'Isotonico': 9.50, 'Suco de fruta': 7.25}
}

# criando as listas de produtos e categorias
produtos = []
categorias = []
precos = []

# preenchendo as listas de produtos, categorias e preços aleatoriamente
for _ in range(n):
    categoria = r.choice(lista_categorias)
    produto = r.choice(list(lista_produtos_categoria[categoria].keys()))
    preco = lista_produtos_categoria[categoria][produto]

    categorias.append(categoria)
    produtos.append(produto)
    precos.append(preco)

# gerando as quantidades dos produtos aleatoriamente
quantidades = [r.randint(1, 10) for _ in range(n)]

### Criação do dataset e transformando em um pandas dataframe

In [10]:
# criando o dataset
dataset = {
    'ID': range(1, n + 1),
    'Data': datas,
    'Produto': produtos,
    'Categoria': categorias,
    'Quantidade': quantidades,
    'Preco': precos
}

# transformando o dataset em um pandas dataframe
df = pd.DataFrame(dataset)

# ordenando o dataframe por ordem de data
df = df.sort_values(by='Data').reset_index(drop=True)

# reorganizando os IDs
df['ID'] = range(1, len(df) + 1)

df

,ID,Data,Produto,Categoria,Quantidade,Preco
0,1,2023-01-01,Bala de gelatina,Conveniencia,4,11.9
1,2,2023-01-02,Dipirona,Remedios,2,7.6
2,3,2023-01-02,Batom,Produtos de beleza,3,25.5
3,4,2023-01-02,Ibuprofeno,Remedios,1,12.6
4,5,2023-01-02,Chocolate,Conveniencia,6,8.7
...,...,...,...,...,...,...
995,996,2023-12-28,Hidratante labial,Produtos de beleza,1,19.9
996,997,2023-12-28,Chocolate,Conveniencia,1,8.7
997,998,2023-12-29,Acetona,Produtos de beleza,3,9.9
998,999,2023-12-30,Hidratante labial,Produtos de beleza,6,19.9


### Simulação de erros no dataset

In [11]:
# criando aleatoriamente valores ausentes na coluna "quantidade" em 2% das linhas do dataframe
nulos = df.sample(frac=0.02, random_state=59).index
df.loc[nulos, 'Quantidade'] = np.nan

# duplicando aleatoriamente 1% das linhas do dataframe
duplicados = df.sample(frac=0.01, random_state=23).index
df_duplicados = df.loc[duplicados]
df = pd.concat([df, df_duplicados], ignore_index=True)

df.info()
print(f'Quantidade de registros: {df["ID"].count()}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010 entries, 0 to 1009
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          1010 non-null   int64  
 1   Data        1010 non-null   object 
 2   Produto     1010 non-null   object 
 3   Categoria   1010 non-null   object 
 4   Quantidade  990 non-null    float64
 5   Preco       1010 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 47.5+ KB
Quantidade de registros: 1010


### Realização da limpeza dos dados

In [12]:
# alterando o tipo da coluna 'data' para datetime
df['Data'] = pd.to_datetime(df['Data'])

# removendo os registros que contêm o valor da coluna 'quantidade' nulo
df = df.dropna(subset=['Quantidade'])

# alterando o tipo da coluna 'quantidade' para inteiro
df['Quantidade'] = df['Quantidade'].astype(int)

# removendo os registros duplicados (mantendo a última ocorrência)
df = df.drop_duplicates(keep='last')

# organizando novamente o dataframe por ordem de data
df = df.sort_values(by='Data').reset_index(drop=True)

# reorganizando novamente os IDs
df['ID'] = range(1, len(df) + 1)

df.info()
print(f'Quantidade de registros: {df["ID"].count()}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 980 entries, 0 to 979
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ID          980 non-null    int64         
 1   Data        980 non-null    datetime64[ns]
 2   Produto     980 non-null    object        
 3   Categoria   980 non-null    object        
 4   Quantidade  980 non-null    int64         
 5   Preco       980 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 46.1+ KB
Quantidade de registros: 980


In [13]:
df

,ID,Data,Produto,Categoria,Quantidade,Preco
0,1,2023-01-01,Bala de gelatina,Conveniencia,4,11.9
1,2,2023-01-02,Dipirona,Remedios,2,7.6
2,3,2023-01-02,Batom,Produtos de beleza,3,25.5
3,4,2023-01-02,Ibuprofeno,Remedios,1,12.6
4,5,2023-01-02,Chocolate,Conveniencia,6,8.7
...,...,...,...,...,...,...
975,976,2023-12-28,Aspirina,Remedios,6,11.5
976,977,2023-12-28,Escova de dente,Higiene pessoal,7,20.5
977,978,2023-12-29,Acetona,Produtos de beleza,3,9.9
978,979,2023-12-30,Hidratante labial,Produtos de beleza,6,19.9


### Salvando o dataframe em um arquivo CSV

In [14]:
# salvando o caminho do notebook na variável path
path = os.getcwd()

# salvando o dataframe em um arquivo CSV na mesma pasta do notebook
df.to_csv(os.path.join(path, 'data_clean.csv'), index=False)

### Breve análise dos dados

In [15]:
# calculando o valor total de vendas por produto
df['Valor total'] = df['Preco'] * df['Quantidade']
df.groupby('Produto')['Valor total'].sum().reset_index(name='Valor total (R$)').sort_values(by='Valor total (R$)', ascending=False)

,Produto,Valor total (R$)
16,Protetor solar,13230.00
12,Locao para acne,11016.00
13,Naproxeno,6570.00
7,Escova de dente,6129.50
3,Batom,5584.50
6,Energetico,4324.80
18,Sabonete liquido,3932.25
8,Fio dental,3859.10
2,Bala de gelatina,3748.50
1,Aspirina,3369.50


In [16]:
# identificando o produto que tem a maior quantidade de unidades vendidas
df.groupby('Produto')['Quantidade'].sum().reset_index(name='Unidades').sort_values(by='Unidades', ascending=False)

,Produto,Unidades
19,Suco de fruta,353
12,Locao para acne,324
18,Sabonete liquido,321
2,Bala de gelatina,315
17,Sabonete em barra,306
13,Naproxeno,300
7,Escova de dente,299
14,Paracetamol,295
1,Aspirina,293
6,Energetico,272


In [17]:
# identificando o produto que tem a maior quantidade de vendas
df.groupby('Produto')['Quantidade'].count().reset_index(name='Vendas').sort_values('Vendas', ascending=False)

,Produto,Vendas
2,Bala de gelatina,68
17,Sabonete em barra,57
18,Sabonete liquido,57
13,Naproxeno,54
14,Paracetamol,54
19,Suco de fruta,54
12,Locao para acne,54
7,Escova de dente,52
1,Aspirina,51
6,Energetico,49
